## Under pressure

In [1]:
from pathlib import Path
import numpy as np
import re
import itertools

In [2]:
pattern = 'Valve (.+) has flow rate=(\d+); tunnels? leads? to valves? (.+)'
pattern = re.compile(pattern)

In [3]:
valves = Path('valves.txt').read_text().split('\n')

valve_map = {}

for line in valves:
    [key, flow, stations] = re.match(pattern, line).groups()
    
    valve_map[key] = [stations.split(', '), int(flow)]

# Identify valves with non-zero flow and order them from largest to lowest flow
non_zero_valves = [key for key in valve_map if valve_map[key][1] > 0]
non_zero_valves = sorted(non_zero_valves, key = lambda x: valve_map[x][1], reverse = True)

In [4]:
def find_shortest_path(start, end):
    paths = [[start]]
    visited = [start]
    
    if start == end:
        return [start]
    
    while end not in visited:
        new_paths = []
        
        for path in paths:
            last = path[-1]
            routes = valve_map[last][0]
            
            
            for route in routes:
                if route in path:
                    continue
                
                else:
                    if route not in visited:
                        visited += [route]
                    
                    new_path = path + [route]
                
                    new_paths += [new_path]
                    
                    if route == end:
                        return new_path
        
        paths = new_paths
                
    return None

In [5]:
class decision():
    
    def __init__(self, location, valves = [], pressures = [], total_pressure = 0, time = 0, prev_decision = None):
        
        self.location = location
        
        self.valves_open = valves
        self.pressures_open = pressures
        self.total_pressure_released = total_pressure
        self.total_pressure_released += np.sum(pressures)
        
        self.time = time
        
        self.prev_decision = prev_decision
        
    def __repr__(self):
        if self.prev_decision == None:
            return f'Start at {self.location}'
        
        elif len(self.prev_decision.valves_open) == len(non_zero_valves):
            return 'Do nothing'
            
        elif self.location != self.prev_decision.location:
            return f'Go to {self.location} from {self.prev_decision.location}'
        
        elif self.location == self.prev_decision.location and self.location in self.valves_open:
            return f'Open valve at {self.location}'
        
        
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        if not isinstance(other, decision):
            return False
        else:
            return (self.location == other.location and self.valves_open == other.valves_open and self.pressures_open == other.pressures_open 
                    and self.total_pressure_released == other.total_pressure_released and self.time == other.time)
            
    
    def make_decisions(self):
        
        valve_open = self.location in self.valves_open
        
        stations, flow = valve_map[self.location]
        
        possible_decisions = []
        
        remaining_valves = np.setdiff1d(non_zero_valves, self.valves_open)
        
        if len(remaining_valves) == 0: # All valves open, do nothing
            return [decision(self.location, self.valves_open, self.pressures_open, self.total_pressure_released, self.time+1, self)]
        
        # Consider opening valve
        elif not valve_open and flow > 0:
            new_valves_open = self.valves_open + [self.location]
            new_pressures_open = self.pressures_open + [flow]
            
            # If this valve is the highest priority remaining valve, just open it:
            if remaining_valves[0] == self.location:
                return [decision(self.location, valves = new_valves_open, 
                                           pressures = new_pressures_open, 
                                           total_pressure = self.total_pressure_released,
                                           time = self.time+1, prev_decision = self)]
            
            else:
                possible_decisions += [decision(self.location, valves = new_valves_open, 
                                           pressures = new_pressures_open, 
                                           total_pressure = self.total_pressure_released,
                                           time = self.time+1, prev_decision = self)]
            
        # Path towards closed valves
        else:
            destinations = [find_shortest_path(self.location, valve)[1] for valve in remaining_valves if valve != self.location]
            destinations = set(destinations)
                            
            possible_decisions += [decision(dest, self.valves_open, self.pressures_open, self.total_pressure_released, self.time+1, self) for dest in destinations]
            
        return possible_decisions

In [298]:
minutes = 30

decisions = {}

decisions[0] = [decision('AA')]

for i in range(minutes):
    current_decisions = decisions[i]
    
    decisions[i+1] = []
    
    for d in current_decisions:
        decisions[i+1] += [dec for dec in d.make_decisions() if dec not in decisions[i+1]]
        
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [6]:
from math import factorial